In [82]:
from collections import Counter
import dotenv, os, requests
import pandas as pd
import numpy as np
import pandasql as ps

In [3]:
dotenv.load_dotenv('ref/.env')

True

In [4]:
league_id = os.environ.get('tds')
sports_data_key = os.environ.get('sports_data')

In [5]:
base_api_url = 'https://api.sleeper.app/v1/'
sports_data_url = 'https://api.sportsdata.io/v3/nfl/stats/json/Injuries/2024'

In [6]:
league_info = requests.get(f'{base_api_url}league/{league_id}').json()

In [7]:
league_info = {
    'league_name' : league_info['name']
    ,'start_week' : league_info['settings']['start_week']
    ,'playoff_start_week' : league_info['settings']['playoff_week_start']
    ,'roster_slots' : [pos for pos in league_info['roster_positions'] if pos != 'BN']
}

In [8]:
def get_injuries_df():
    injuries_resp = [requests.get(f'{sports_data_url}/{wk}?key={sports_data_key}').json() for wk in range(league_info['start_week'], league_info['playoff_start_week'])]
    injuries = []
    for wk in injuries_resp:
        for injury in wk:
            injuries.append({'full_name' : injury['Name'], 'week' : injury['Week'], 'team' : injury['Team']})
    return pd.DataFrame(injuries)

In [9]:
injuries_df = get_injuries_df()

In [10]:
def get_weekly_matchups_dfs():
    plyr_wkly_points_dicts = []
    wkly_matchups_dicts = []
    wkly_matchups = [requests.get(f'{base_api_url}league/{league_id}/matchups/{wk}').json() for wk in range(league_info['start_week'], league_info['playoff_start_week'])]
    week_num = 0
    for wk in wkly_matchups:
        week_num += 1
        for matchup in wk:
            wkly_matchups_dicts.append({'roster_id' : matchup['roster_id'], 'week' : week_num, 'matchup_id' : matchup['matchup_id'], 'ttl_points' : matchup['points']})
            for player in matchup['players']:
                plyr_wkly_points_dicts.append({'roster_id': matchup['roster_id'], 'player_id' : player, 'week': week_num, 'is_starter' : True if player in matchup['starters'] else False, 'points' : matchup['players_points'][player]})
    return (pd.DataFrame(wkly_matchups_dicts), pd.DataFrame(plyr_wkly_points_dicts))

In [11]:
def get_weekly_transactions_dfs():
    wkly_trnsctns = [requests.get(f'{base_api_url}league/{league_id}/transactions/{wk}').json() for wk in range(league_info['start_week'], league_info['playoff_start_week'])]
    trns_dicts = []
    plyr_trns_dicts = []
    week_num = 0
    player_trns_typs = ['add', 'drop']
    for wk in wkly_trnsctns:
        week_num += 1
        for trns in wk:
            for trns_typ in player_trns_typs:
                if trns[f'{trns_typ}s']:
                    for player in trns[f'{trns_typ}s'].keys():
                        plyr_trns_dicts.append({
                            'week' : week_num
                            ,'player_id' : player
                            ,'roster_id' : trns[f'{trns_typ}s'][player]
                            ,'add_drop' : trns_typ
                            ,'trns_id' : trns['transaction_id']
                            ,'create_tms' : trns['created']
                        })
            for roster in trns['roster_ids']:
                trns_dicts.append({
                    'roster_id' : roster
                    ,'trns_typ' : trns['type']
                    ,'trns_status' : trns['status']
                    ,'trns_id' : trns['transaction_id']
                    ,'create_tms' : trns['created']
                })
    return (pd.DataFrame(trns_dicts), pd.DataFrame(plyr_trns_dicts))

In [12]:
def get_roster_id_by_team_df():
    users_df = pd.DataFrame(requests.get(f'{base_api_url}league/{league_id}/users').json())[['display_name', 'user_id']]
    rosters_df = pd.DataFrame([{'roster_id' : roster['roster_id'], 'owner_id' : roster['owner_id'], 'record' : roster['metadata']['record']} for roster in requests.get(f'{base_api_url}league/{league_id}/rosters').json()])
    return users_df.merge(rosters_df, how='inner', left_on='user_id', right_on='owner_id')[['display_name', 'roster_id', 'record']]

In [13]:
def get_draft_df():
    draft_id = requests.get(f'{base_api_url}league/{league_id}/drafts').json()[0]['draft_id']
    draft_tm = requests.get(f'{base_api_url}draft/{draft_id}').json()['created']
    draft_resp = requests.get(f'{base_api_url}draft/{draft_id}/picks').json()
    return pd.DataFrame([{'player_id' : pick['player_id'], 'roster_id' : pick['roster_id'], 'round' : pick['round'], 'trns_typ' : 'draft', 'trns_status' : 'complete', 'week' : 0, 'add_drop' : 'add', 'create_tms' : draft_tm, 'trns_id' : f'{pick['draft_id']}{pick['roster_id']}{pick['round']}'} for pick in draft_resp])

In [14]:
draft_df = get_draft_df()

In [15]:
matchups_df, plyr_points_df = get_weekly_matchups_dfs()

In [16]:
trns_df, plyr_trns_df = get_weekly_transactions_dfs()

In [17]:
draft_df['create_tms'] = pd.to_datetime(draft_df.create_tms, unit='ms')
plyr_trns_df['create_tms'] = pd.to_datetime(plyr_trns_df.create_tms, unit='ms')
trns_df['create_tms'] = pd.to_datetime(trns_df.create_tms, unit='ms')

In [18]:
plyr_trns_df = pd.concat([plyr_trns_df, draft_df[['week', 'player_id', 'roster_id', 'add_drop', 'trns_id', 'create_tms']]])
trns_df = pd.concat([trns_df, draft_df[['roster_id', 'trns_typ', 'trns_status', 'trns_id', 'create_tms']]])

In [19]:
plyr_df = pd.read_json('ref/players.json', orient='index')[['first_name', 'last_name', 'team', 'position', 'player_id', 'years_exp', 'full_name']]

/var/folders/0b/_drclt_54kb8why6nm6znsnh0000gn/T/ipykernel_91827/2706744226.py:1: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  plyr_df = pd.read_json('ref/players.json', orient='index')[['first_name', 'last_name', 'team', 'position', 'player_id', 'years_exp', 'full_name']]
/var/folders/0b/_drclt_54kb8why6nm6znsnh0000gn/T/ipykernel_91827/2706744226.py:1: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  plyr_df = pd.read_json('ref/players.json', orient='index')[['first_name', 'last_nam

In [20]:
rosters_df = get_roster_id_by_team_df()

In [301]:
def get_rookie_starts_df(plyr_points_df, plyr_df, rosters_df):
    strted_df = plyr_points_df[plyr_points_df.is_starter == True].merge(plyr_df[plyr_df.years_exp == 0], on='player_id', how='inner').merge(rosters_df, on='roster_id', how='inner')[['player_id', 'display_name', 'week']]
    rookies_strtd_df = strted_df.groupby('display_name').player_id.nunique().to_frame().reset_index().rename({'player_id' : 'num_rookies_started'}, axis=1)
    rookie_strts_df = strted_df.groupby('display_name').player_id.count().to_frame().reset_index().rename({'player_id' : 'ttl_rookie_starts'}, axis=1)
    return rookies_strtd_df.merge(rookie_strts_df).sort_values(by=['ttl_rookie_starts', 'num_rookies_started'], ascending=False).reset_index(drop=True)

In [22]:
def get_win_streak(row):
    return max(map(len, row.split('L')))

def get_loss_streak(row):
    return max(map(len, row.split('W')))

In [23]:
rosters_df['win_streak'] = rosters_df.record.apply(get_win_streak)
rosters_df['loss_streak'] = rosters_df.record.apply(get_loss_streak)

In [24]:
def get_rstrd_plyr_to_fm_df(plyr_trns_df, trns_df):
    merged_trns_df = plyr_trns_df.merge(trns_df[trns_df.trns_status == 'complete'], how='inner').sort_values(['roster_id', 'player_id', 'create_tms'])
    merged_trns_df['plyr_actn_rnk'] = merged_trns_df.groupby(['player_id', 'roster_id'])['create_tms'].rank(ascending=True)
    merged_trns_df.loc[merged_trns_df.add_drop == 'add', 'merge_on_actn_rnk'] = merged_trns_df.plyr_actn_rnk + 1
    merged_trns_df.drop(['trns_id', 'create_tms', 'trns_status'], axis=1, inplace=True)
    merged_trns_df = merged_trns_df.merge(merged_trns_df, left_on=['merge_on_actn_rnk', 'player_id', 'roster_id'], right_on=['plyr_actn_rnk', 'player_id', 'roster_id'], how='left')
    merged_trns_df = merged_trns_df[merged_trns_df.add_drop_x == 'add']
    merged_trns_df = merged_trns_df.rename({
        'week_x' : 'week_fm'
        ,'week_y' : 'week_to'
        ,'trns_typ_x' : 'add_action'
        ,'trns_typ_y' : 'drop_action'
    }, axis=1).drop(['add_drop_x', 'add_drop_y', 'plyr_actn_rnk_x', 'plyr_actn_rnk_y', 'merge_on_actn_rnk_x', 'merge_on_actn_rnk_y'], axis=1)
    merged_trns_df['week_to'] = merged_trns_df.week_to.fillna(league_info['playoff_start_week']).astype('int')
    return merged_trns_df[(merged_trns_df.add_action != 'commissioner') | (merged_trns_df.week_fm != 1)][['roster_id', 'player_id', 'add_action', 'drop_action', 'week_fm', 'week_to']]

In [25]:
rstrd_plyr_to_fm_df = get_rstrd_plyr_to_fm_df(plyr_trns_df, trns_df)

In [208]:
def get_waiver_scores_df(rstrd_plyr_to_fm_df, plyr_points_df):
    return ps.sqldf("""
    SELECT
        rdf.display_name
        ,pdf.first_name
        ,pdf.last_name
        ,ppdf.player_id
        ,Sum(ppdf.points) AS ttl_points
        ,count(ppdf.week) AS wks_strtd
        ,Sum(ppdf.points) / Count(ppdf.week) AS avg_strt_pts
        FROM plyr_points_df ppdf
        INNER JOIN rstrd_plyr_to_fm_df rpdf
            ON rpdf.player_id = ppdf.player_id
            AND rpdf.roster_id = ppdf.roster_id
            AND ppdf.week >= rpdf.week_fm
            AND ppdf.week < rpdf.week_to
        INNER JOIN rosters_df rdf
            ON rdf.roster_id = rpdf.roster_id
        INNER JOIN plyr_df pdf
            ON pdf.player_id = ppdf.player_id
        WHERE 1=1
            AND ppdf.is_starter = True
            AND rpdf.drop_action IN ('waiver', 'free_agent')
            AND rpdf.add_action IN ('waiver', 'free_agent')
        GROUP BY 1,2
    """)

In [224]:
def get_wkly_matchup_results_df(matchups_df, rosters_df):
    wk_median_df = pd.DataFrame(matchups_df.groupby(matchups_df.week)[['ttl_points']].median()).reset_index().rename({'ttl_points' : 'wk_median'}, axis=1)
    return ps.sqldf("""
    SELECT
        rdf.display_name
        ,l.week
        ,l.matchup_id
        ,l.ttl_points AS pf
        ,r.ttl_points AS pa
        ,ABS(l.ttl_points - r.ttl_points) AS abs_pt_diff
        ,wmdf.wk_median AS league_median
        ,CASE
            WHEN l.ttl_points > r.ttl_points
                THEN 1
            ELSE 0
        END AS win
        ,CASE
            WHEN l.ttl_points < r.ttl_points
                THEN 1
            ELSE 0
        END AS loss
        ,CASE
            WHEN l.ttl_points = r.ttl_points
                THEN 1
            ELSE 0
        END AS tie
        ,CASE
            WHEN l.ttl_points > wmdf.wk_median
                THEN 1
            ELSE 0
        END AS median_win
        ,CASE
            WHEN l.ttl_points < wmdf.wk_median
                THEN 1
            ELSE 0
        END AS median_loss
        ,CASE
            WHEN l.ttl_points = wmdf.wk_median
                THEN 1
            ELSE 0
        END AS median_tie
        FROM matchups_df l
        INNER JOIN matchups_df r
            ON r.matchup_id = l.matchup_id
            AND r.roster_id <> l.roster_id
            AND r.week = l.week
        INNER JOIN rosters_df rdf
            ON rdf.roster_id = l.roster_id
        INNER JOIN wk_median_df wmdf
            ON wmdf.week = l.week
    """)

In [225]:
matchup_results_df = get_wkly_matchup_results_df(matchups_df, rosters_df)

In [29]:
srtd_wins_df = matchup_results_df[matchup_results_df.win == 1].sort_values('abs_pt_diff', ascending=False)
srtd_losses_df = matchup_results_df[matchup_results_df.loss == 1].sort_values('abs_pt_diff', ascending=False)

In [30]:
median_results_df = matchup_results_df.groupby('display_name')[['pf', 'pa', 'win', 'loss', 'tie', 'median_win', 'median_loss', 'median_tie']].sum().reset_index()

In [47]:
median_results_df['win_perc'] = (median_results_df.win + (median_results_df.tie / 2)) / (median_results_df.win + median_results_df.loss + median_results_df.tie)
median_results_df['ttl_wins'] = median_results_df.win + median_results_df.median_win
median_results_df['ttl_losses'] = median_results_df.loss + median_results_df.median_loss
median_results_df['ttl_ties'] = median_results_df.tie + median_results_df.median_tie
median_results_df['ttl_win_perc'] = (median_results_df.ttl_wins + (median_results_df.ttl_ties / 2)) / (median_results_df.ttl_wins + median_results_df.ttl_losses + median_results_df.ttl_ties)
median_results_df['win_perc_diff'] = median_results_df.ttl_win_perc - median_results_df.win_perc

In [38]:
rosterd_injuries = ps.sqldf("""
SELECT
    rdf.display_name
    ,pdf.player_id
    ,idf.week
    ,idf.team
    FROM injuries_df idf
    INNER JOIN plyr_df pdf
        ON pdf.full_name = idf.full_name
        AND pdf.team = idf.team
    INNER JOIN rstrd_plyr_to_fm_df rptfdf
        ON rptfdf.player_id = pdf.player_id
        AND idf.week BETWEEN rptfdf.week_fm AND rptfdf.week_to
    INNER JOIN rosters_df rdf
        ON rdf.roster_id = rptfdf.roster_id
""").drop_duplicates()

In [ ]:
roster_pos_df = pd.DataFrame([{'position' : pos, 'cnt' : league_info['roster_slots'].count(pos)} for pos in set(league_info['roster_slots'])])

In [146]:
wkly_plyr_scores_df = ps.sqldf("""
SELECT
    ppdf.roster_id
    ,pdf.player_id
    ,ppdf.week
    ,ppdf.points
    ,pdf.position
    ,RANK() OVER (PARTITION BY ppdf.roster_id, ppdf.week, pdf.position ORDER BY ppdf.points DESC) AS pos_rnk
    ,rpdf.cnt AS pos_roster_slots
    FROM plyr_points_df ppdf
    INNER JOIN plyr_df pdf
        ON pdf.player_id = ppdf.player_id
    LEFT JOIN roster_pos_df rpdf
        ON rpdf.position = pdf.position
""")

In [148]:
flex_rnk_df = ps.sqldf(f""" 
SELECT
    roster_id
    ,player_id
    ,week
    ,points
    ,'FLEX' AS position
    ,RANK() OVER (PARTITION BY roster_id, week ORDER BY points DESC) AS pos_rnk
    ,rpdf.cnt AS pos_roster_slots
    FROM wkly_plyr_scores_df wpsdf
    LEFT JOIN roster_pos_df rpdf
        ON rpdf.position = 'FLEX'
    WHERE 1=1
        AND pos_rnk > pos_roster_slots
        AND wpsdf.position IN ('WR', 'RB', 'TE')
""")

In [170]:
max_wkly_points_df = ps.sqldf("""
SELECT
    roster_id
    ,week
    ,ROUND(SUM(points), 2) as max_points
    FROM
        (SELECT
            roster_id
            ,week
            ,points
            FROM wkly_plyr_scores_df
            WHERE 1=1
                AND pos_rnk <= pos_roster_slots
        UNION ALL
        SELECT
            roster_id
            ,week
            ,points
            FROM flex_rnk_df
            WHERE 1=1
                AND pos_rnk <= pos_roster_slots
        )
    GROUP BY 1,2
""")

In [171]:
matchups_max_points_df = matchups_df.merge(max_wkly_points_df, how='inner', on=['roster_id', 'week'])

In [180]:
matchups_max_points_df['point_diff'] = (matchups_max_points_df.max_points - matchups_max_points_df.ttl_points).round(2)
matchups_max_points_df['strt_eff'] = matchups_max_points_df.ttl_points / matchups_max_points_df.max_points

In [210]:
waiver_scores_df = get_waiver_scores_df(rstrd_plyr_to_fm_df, plyr_points_df)

In [313]:
big_red_too_drunk_df = ps.sqldf("""
SELECT
    l.week
    ,l.display_name AS winner
    ,l.pf AS winner_pf
    ,r.display_name AS loser
    ,r.pf AS loser_pf
    ,l.abs_pt_diff
    FROM matchup_results_df l
    INNER JOIN matchup_results_df r
        ON r.matchup_id = l.matchup_id
        AND r.week = l.week
        AND l.win = 1
        AND r.loss = 1
    ORDER BY 6 DESC
""")

In [314]:
break_it_df = big_red_too_drunk_df.sort_values(by='loser_pf', ascending=False).reset_index()

In [246]:
chubby_df = matchup_results_df[matchup_results_df.win == 1].groupby('display_name')[['abs_pt_diff']].mean().sort_values(by='abs_pt_diff', ascending=False).reset_index()

In [257]:
evil_on_me_df = rosterd_injuries.groupby(['display_name'])['player_id'].nunique().reset_index().sort_values(by='player_id', ascending=False).reset_index(drop=True)

In [297]:
drive_with_your_heart_df = ps.sqldf("""
SELECT
    display_name
    ,ROUND(AVG(strt_eff), 4) AS avg_strt_eff
    FROM matchups_max_points_df mmpdf
    INNER JOIN rosters_df rdf
        ON rdf.roster_id = mmpdf.roster_id
    GROUP BY 1
    ORDER BY 2 DESC
""")

In [275]:
hakuna_matata_df = matchup_results_df.groupby('display_name')['pf'].max().reset_index().sort_values(by='pf', ascending=False).reset_index(drop=True)

In [278]:
ice_storm_df = matchup_results_df.groupby('display_name')['pa'].sum().reset_index().sort_values(by='pa', ascending=False)

In [294]:
target_df = waiver_scores_df.sort_values(by='ttl_points', ascending=False)[['display_name', 'first_name', 'last_name', 'ttl_points', 'wks_strtd', 'avg_strt_pts']].reset_index(drop=True).round(2)

In [292]:
excellence_df = matchup_results_df.groupby('display_name')['pf'].mean().reset_index().sort_values(by='pf', ascending=False).reset_index(drop=True).round(2)

In [299]:
on_fire_df = rosters_df.sort_values(by='win_streak', ascending=False)[['display_name', 'win_streak']].reset_index(drop=True)

In [302]:
ten_yo_df = get_rookie_starts_df(plyr_points_df, plyr_df, rosters_df)

In [304]:
spider_monkey_df = matchup_results_df.groupby('display_name')['pf'].sum().reset_index().sort_values(by='pf', ascending=False).reset_index(drop=True)

In [308]:
magic_man_df = matchup_results_df[matchup_results_df.win == 1].groupby('display_name')[['abs_pt_diff']].mean().sort_values(by='abs_pt_diff', ascending=True).reset_index()

In [310]:
not_good_df = rosters_df.sort_values(by='loss_streak', ascending=False)[['display_name', 'loss_streak']].reset_index(drop=True)

In [320]:
man_df = ps.sqldf("""
SELECT
    rdf.display_name
    ,AVG(pdf.years_exp) AS avg_strtr_yrs_exp
    FROM plyr_points_df ppdf
    INNER JOIN plyr_df pdf
        ON pdf.player_id = ppdf.player_id
        AND pdf.position <> 'DEF'
    INNER JOIN rosters_df rdf
        ON rdf.roster_id = ppdf.roster_id
    WHERE 1=1
        AND ppdf.is_starter = True
    GROUP BY 1
    ORDER BY 2 DESC
""")

In [322]:
wild_horses_df = matchup_results_df.groupby('display_name')['pf'].min().reset_index().sort_values(by='pf', ascending=True).reset_index(drop=True)

In [329]:
median_results_df = ps.sqldf("""
SELECT
    display_name
    ,pf
    ,pa
    ,win
    ,loss
    ,tie
    ,win_perc
    ,RANK() OVER (ORDER BY win_perc DESC, pf DESC) AS rnk
    ,median_win
    ,median_loss
    ,median_tie
    ,ttl_wins
    ,ttl_losses
    ,ttl_ties
    ,ttl_win_perc
    ,RANK() OVER (ORDER BY ttl_win_perc DESC, pf DESC) AS ttl_rnk
    ,win_perc_diff
    ,RANK() OVER (ORDER BY win_perc DESC, pf DESC) - RANK() OVER (ORDER BY ttl_win_perc DESC, pf DESC) AS rnk_diff
    FROM median_results_df
""")

In [ ]:
dfs = {
    'blowout_win_loss' : big_red_too_drunk_df
    ,'highest_avg_mov' : chubby_df
    ,'injuries' : evil_on_me_df
    ,'efficiency' : drive_with_your_heart_df
    ,'highest_score_wks' : hakuna_matata_df
    ,'pa' : ice_storm_df
    ,'waiver_adds' : target_df
    ,'avg_wkly_scores' : excellence_df
    ,'win_streaks' : on_fire_df
    ,'strtd_rookies' : ten_yo_df
    ,'pf' : spider_monkey_df
    ,'lowest_avg_mov' : magic_man_df
    ,'loss_streaks' : not_good_df
    ,'strtr_exp' : man_df
    ,'lowest_score_wks' : wild_horses_df
    ,'median_rcrds' : median_results_df
}